## Transformer NeuronX를 이용해 모델 컴파일 

가장 먼저 이용할 수 있는 옵션은, Transformer NeuronX 라이브러리의 [컴파일](https://awsdocs-neuron.readthedocs-hosted.com/en/latest/compiler/neuronx-cc/api-reference-guide/neuron-compiler-cli-reference-guide.html#neuron-compiler-cli-reference-guide) 기능을 이용해 모델을 컴파일 하는 것입니다. 성능을 개선하기 위해 몇 가지 추가 구성을 설정할 수 있습니다.

* attention_layout: Attention 계산에 사용할 레이아웃으로 "BSH"를 사용합니다.
* fuse_qkv: QKV 투영을 단일 행렬 곱셈으로 융합하여 Q/K/V 가중치 로딩 효율성을 높입니다.
* group_query_attention: KV 캐시 샤딩 전략에 대한 자세한 내용은 transformers neuronx의 Grouped Query Attention을 참조하세요. (See Appendix. 4)
* quant: 모델의int8 가중치 저장은 MLP 레이어의 로드 시간을 개선할 수 있습니다. 
* on_device_generation: Transformers Neuron은 장치에서 top-k 샘플링을 지원하며, 이는 성능을 향상시키는 데 도움이 됩니다.

Llama 3 모델의 사용은 Meta 라이선스에 의해 규제되며, 이 샘플을 실행하기 전에 다운로드해야 합니다. Meta로부터 Llama 3 모델에 접근하기 위한 단계는 [meta-llama/Meta-Llama-3-8B](https://huggingface.co/meta-llama/Meta-Llama-3-8B)에 설명되어 있습니다.

이 노트북을 실행하기 위해 [meta-llama/Meta-Llama-3-8B](https://huggingface.co/meta-llama/Meta-Llama-3-8B) 모델을 Hugging Face에서 접근할 수 있어야 합니다. 이후 HuggingFace에 로그인합니다.

In [3]:
from huggingface_hub import login
login()

In [ ]:
모델을 컴파일 하기 전, 해당 모델의 적정 batch_size와 tp_degree, n_positions를 확인한 후 아래 코드를 업데이트 합니다.

In [ ]:
import time
import torch
from transformers import AutoTokenizer
from transformers_neuronx import LlamaForSampling
from transformers import LlamaForCausalLM, LlamaTokenizer, PreTrainedTokenizerFast
from transformers_neuronx import LlamaForSampling, NeuronConfig, GQA, QuantizationConfig
from transformers_neuronx.config import GenerationConfig 

# Hugging Face 모델 ID 설정
model_id = "meta-llama/Meta-Llama-3-8B/"

neuron_config = NeuronConfig(
                    on_device_embedding=False,
                    attention_layout='BSH',
                    fuse_qkv=True,
                    group_query_attention=GQA.REPLICATED_HEADS,
                    quant=QuantizationConfig(quant_dtype='s8', dequant_dtype='f16'),
                    on_device_generation=GenerationConfig(do_sample=True)
              )

# meta-llama/Meta-Llama-3-8B 모델을 NeuronCores에 12-way 텐서 병렬 처리로 로드하고 컴파일 실행
neuron_model = LlamaForSampling.from_pretrained(model_id, neuron_config=neuron_config, batch_size=1, tp_degree=12, amp='f16', n_positions=8192)
neuron_model.to_neuron() # 모델 로드/컴파일

Neuron LlamaForSampling 클래스를 사용하여 Llama 3 모델에 텐서 병렬 처리를 구현하는 방법에 대해 알아보겠습니다. 이 기술을 적용하려면, 모델의 시나리오에 맞는 시퀀스 길이로 `n_positions`를 설정해야 합니다. 이 값은 예상되는 입력 및 출력 토큰의 합보다 반드시 커야 합니다. 텐서 병렬 처리는 `tp_degree=12` 인수를 통해 활성화됩니다.

성능을 높이기 위해 `amp='f16'` 플래그를 사용하여 모델을 float16으로 캐스팅할 수 있습니다. 이렇게 하면 모델의 계산 효율성이 향상되어, 리소스 사용을 최적화하며 더 빠른 추론 속도를 얻을 수 있습니다.

마지막으로, 모델의 계산 그래프는 Neuron에서 최적화된 추론을 위해 neuronx-cc에 의해 컴파일됩니다. 이 과정을 통해 모델은 AWS Neuron 하드웨어에서 최적의 성능을 발휘할 수 있도록 준비됩니다. 이러한 접근 방식은 딥러닝 모델의 효율과 성능을 극대화하는 데 큰 도움이 됩니다.

모델을 한 번 컴파일하고 나면, 해당 결과물을 save 함수를 사용하여 지정된 디렉토리에 저장할 수 있습니다. 이렇게 저장된 모델 아티팩트는 나중에 load 함수를 통해 쉽게 불러올 수 있으며, 이 과정은 모델을 다시 컴파일할 필요 없이 빠르게 모델을 배포할 수 있게 해 줍니다.

In [ ]:
neuron_model.save('./neuron_artifacts') # can be copied and used on a different neuron instance

컴파일이 완료되면, 다음과 같은 형태의 디렉토리 구조가 완성이 됩니다.

```plaintext
neuron_artifacts/
├── 14c0de162adad18e6e77.neff
├── 1cabdc562e64e63d0081.neff
├── 641d2daf2e65a013406d.neff
├── 7ad05afb0577f47d52db.neff
├── 8159e002a19b6ddfe12f.neff
├── 89e76df03b41e6cc5d22.neff
├── 9340a6b565a68744724c.neff
├── c637f30abbd2f600a421.neff
├── c820947075ac802b30f3.neff
├── ccbc3860b984ace7420a.neff
├── d96191b583dde6135aa9.neff
└── de011594ed2f0cc29300.neff
```